In [1]:
import os, glob
import json
import pandas as pd
import numpy as np

from sedpy.observate import Filter, load_filters
#import prospect

In [29]:
redshifts = pd.read_csv('data/redshifts.tsv', delimiter='\t')
redshifts

,IAU Name,Date,Phase,Telescope/Inst.,TDE Class,Redshift,ID,Unnamed: 7
0,AT2018zr,2018 Mar 28,25,WHT/ISIS^a,TDE-H,0.0710,1,NaN
1,AT2018bsi,2018 May 13,34,DCT/De Veny,TDE-H+He,0.0510,2,NaN
2,AT2018hco,2018 Nov 10,29,Keck/LRIS,TDE-H,0.0880,3,NaN
3,AT2018iih,2019 Mar 10,102,DCT/De Veny,TDE-He,0.2120,4,NaN
4,AT2018hyz,2018 Nov 12,6,FTN/Floyds-N^b,TDE-H,0.0458,5,NaN
5,AT2018lni,2019 Mar 1,81,DCT/De Veny,TDE-H+He,0.1380,6,NaN
6,AT2018lna,2019 Jan 26,0,Palomar/DBSP,TDE-H+He,0.0910,7,NaN
7,AT2019cho,2019 May 2,58,DCT/De Veny,TDE-H+He,0.1930,8,NaN
8,AT2019bhf,2019 May 29,90,DCT/De Veny,TDE-H,0.1206,9,NaN
9,AT2019azh,2019 May 1,46,Keck/LRIS,TDE-H+He,0.0222,10,NaN


# Read in and Reformat Photometry

In [53]:
path = os.path.join(os.getcwd(), 'data', 'tde_catalog_v0.2.json')
with open(path, 'r') as f:
    j = json.load(f)

# get just the observational data for the host galaxies
# and fix the filter names for sedpy
obsdata = {}
filtermap = {'GALEX_FUV': 'galex_FUV',
             'GALEX_NUV': 'galex_NUV',
             'uvot_U': 'uvot_u',
             'uvot_B': 'uvot_b',
             'uvot_V': 'uvot_v'}

for tdename in j:
    out = j[tdename]['host']['observations']
    mask = []
    for i, filt in enumerate(out['filternames']):
        if filt in filtermap:
            out['filternames'][i] = filtermap[filt]
        if 'wise' in filt:
            mask.append(False)
        else:
            mask.append(True)
    out['photmask'] = mask
    
    obsdata[tdename] = out
    
    #print(redshifts['IAU Name'].values)
    if tdename in redshifts['IAU Name'].values:
        z = redshifts.Redshift[redshifts['IAU Name'] == tdename].values
        obsdata[tdename]['redshift'] = float(z[0])
    

In [54]:
obsdata['AT2018lni']

{'maggies': [0.0,
  0.0,
  9.825792309282996e-09,
  1.64499447921247e-08,
  2.517041767693601e-08,
  3.217619440194719e-08,
  3.046839409174361e-08,
  2.7593068558175005e-08,
  1.5674727518004407e-08,
  0.0],
 'maggies_unc': [2.1038554848624914e-09,
  7.531001421802101e-10,
  5.230835934066284e-10,
  7.575479531784273e-10,
  1.1591405705469335e-09,
  1.4817685115840413e-09,
  1.4151882449097501e-09,
  1.4994351646355887e-09,
  2.2954762532929523e-09,
  3.635549015798299e-08],
 'mags': [19.945035227787336,
  21.060443165176927,
  20.019081049792025,
  19.45958888813241,
  18.997773944315078,
  18.731163307053727,
  18.79037608921144,
  18.898000000000003,
  19.511999999999997,
  17.846],
 'mags_unc': [0.05,
  0.05,
  0.0578,
  0.05,
  0.05,
  0.05,
  0.05043,
  0.059,
  0.159,
  0.05],
 'filternames': ['galex_FUV',
  'galex_NUV',
  'ps_g',
  'ps_r',
  'ps_i',
  'ps_z',
  'ps_y',
  'wise_w1',
  'wise_w2',
  'wise_w3'],
 'photmask': [True, True, True, True, True, True, True, False, False,

In [55]:
outdir = os.path.join(os.getcwd(), 'data', 'hostdata')
for tdename in obsdata:
    outfile = os.path.join(outdir, f'{tdename}.json')
    j = json.dumps(obsdata[tdename], indent=4)
    with open(outfile, 'w') as f:
        f.write(j)

# Some code for prospector

In [5]:
run_params = {'verbose': True,
              'debug': False,
              'outfile': 'test',
              'output_pickles': False,
              # Optimization parameters
              'do_powell': False,
              'ftol': 0.5e-5, 'maxfev': 5000,
              'do_levenberg': True,
              'nmin': 10,
              # emcee fitting parameters
              'nwalkers': 128,
              'nburn': [16, 32, 64],
              'niter': 512,
              'interval': 0.25,
              'initial_disp': 0.1,
              # Obs data parameters
              'objid': 0,
              'phottable': 'demo_photometry.dat',
              'luminosity_distance': 1e-5,  # in Mpc
              # Model parameters
              'add_neb': False,
              'add_duste': False,
              # SPS parameters
              'zcontinuous': 1,
              }

In [6]:
def build_model(object_redshift=0.0, fixed_metallicity=None, add_duste=False,
                add_neb=False, luminosity_distance=0.0, **extras):
    """Construct a model.  This method defines a number of parameter
    specification dictionaries and uses them to initialize a
    `models.sedmodel.SedModel` object.

    :param object_redshift:
        If given, given the model redshift to this value.

    :param add_dust: (optional, default: False)
        Switch to add (fixed) parameters relevant for dust emission.

    :param add_neb: (optional, default: False)
        Switch to add (fixed) parameters relevant for nebular emission, and
        turn nebular emission on.

    :param luminosity_distance: (optional)
        If present, add a `"lumdist"` parameter to the model, and set it's
        value (in Mpc) to this.  This allows one to decouple redshift from
        distance, and fit, e.g., absolute magnitudes (by setting
        luminosity_distance to 1e-5 (10pc))
    """
    from prospect.models.templates import TemplateLibrary
    from prospect.models import priors, sedmodel

    # --- Get a basic delay-tau SFH parameter set. ---
    # This has 5 free parameters:
    #   "mass", "logzsol", "dust2", "tage", "tau"
    # And two fixed parameters
    #   "zred"=0.1, "sfh"=4
    # See the python-FSPS documentation for details about most of these
    # parameters.  Also, look at `TemplateLibrary.describe("parametric_sfh")` to
    # view the parameters, their initial values, and the priors in detail.
    model_params = TemplateLibrary["parametric_sfh"]

    # Add lumdist parameter.  If this is not added then the distance is
    # controlled by the "zred" parameter and a WMAP9 cosmology.
    if luminosity_distance > 0:
        model_params["lumdist"] = {"N": 1, "isfree": False,
                                   "init": luminosity_distance, "units":"Mpc"}

    # Adjust model initial values (only important for optimization or emcee)
    model_params["dust2"]["init"] = 0.1
    model_params["logzsol"]["init"] = -0.3
    model_params["tage"]["init"] = 13.
    model_params["mass"]["init"] = 1e8

    # If we are going to be using emcee, it is useful to provide an
    # initial scale for the cloud of walkers (the default is 0.1)
    # For dynesty these can be skipped
    model_params["mass"]["init_disp"] = 1e7
    model_params["tau"]["init_disp"] = 3.0
    model_params["tage"]["init_disp"] = 5.0
    model_params["tage"]["disp_floor"] = 2.0
    model_params["dust2"]["disp_floor"] = 0.1

    # adjust priors
    model_params["dust2"]["prior"] = priors.TopHat(mini=0.0, maxi=2.0)
    model_params["tau"]["prior"] = priors.LogUniform(mini=1e-1, maxi=10)
    model_params["mass"]["prior"] = priors.LogUniform(mini=1e6, maxi=1e10)

    # Change the model parameter specifications based on some keyword arguments
    if fixed_metallicity is not None:
        # make it a fixed parameter
        model_params["logzsol"]["isfree"] = False
        #And use value supplied by fixed_metallicity keyword
        model_params["logzsol"]['init'] = fixed_metallicity

    if object_redshift != 0.0:
        # make sure zred is fixed
        model_params["zred"]['isfree'] = False
        # And set the value to the object_redshift keyword
        model_params["zred"]['init'] = object_redshift

    if add_duste:
        # Add dust emission (with fixed dust SED parameters)
        model_params.update(TemplateLibrary["dust_emission"])

    if add_neb:
        # Add nebular emission (with fixed parameters)
        model_params.update(TemplateLibrary["nebular"])

    # Now instantiate the model using this new dictionary of parameter specifications
    model = sedmodel.SedModel(model_params)

    return model

In [7]:
def build_obs(objid=0, phottable='demo_photometry.dat',
              luminosity_distance=None, **kwargs):
    """Load photometry from an ascii file.  Assumes the following columns:
    `objid`, `filterset`, [`mag0`,....,`magN`] where N >= 11.  The User should
    modify this function (including adding keyword arguments) to read in their
    particular data format and put it in the required dictionary.

    :param objid:
        The object id for the row of the photomotery file to use.  Integer.
        Requires that there be an `objid` column in the ascii file.

    :param phottable:
        Name (and path) of the ascii file containing the photometry.

    :param luminosity_distance: (optional)
        The Johnson 2013 data are given as AB absolute magnitudes.  They can be
        turned into apparent magnitudes by supplying a luminosity distance.

    :returns obs:
        Dictionary of observational data.
    """

    from prospect.utils.obsutils import fix_obs
    
    # Build output dictionary.
    obs = {}
    # This is a list of sedpy filter objects.    See the
    # sedpy.observate.load_filters command for more details on its syntax.
    obs['filters'] = load_filters(tde['filternames'])
    # This is a list of maggies, converted from mags.  It should have the same
    # order as `filters` above.
    obs['maggies'] = np.array(tde['maggies'])
    # HACK.  You should use real flux uncertainties
    obs['maggies_unc'] = np.array(tde['maggies_unc'])
    # Here we mask out any NaNs or infs
    #obs['phot_mask'] = None
    # We have no spectrum.
    obs['wavelength'] = None
    obs['spectrum'] = None

    # Add unessential bonus info.  This will be stored in output
    #obs['dmod'] = catalog[ind]['dmod']
    obs['objid'] = tdename

    # This ensures all required keys are present and adds some extra useful info
    obs = fix_obs(obs)

    return obs

In [8]:
def build_sps(zcontinuous=1, compute_vega_mags=False, **extras):
    from prospect.sources import CSPSpecBasis
    sps = CSPSpecBasis(zcontinuous=zcontinuous,
                       compute_vega_mags=compute_vega_mags)
    return sps

def build_noise(**extras):
    return None, None

def build_all(**kwargs):

    return (build_obs(**kwargs), build_model(**kwargs),
            build_sps(**kwargs), build_noise(**kwargs))


In [9]:
os.getenv('SPS_HOME')

'/home/nfranz/bin/fsps/'

In [24]:
filters = []
for tde in obsdata.values():
    for filt in tde['filternames']:
        if filt not in filters:
            filters.append(filt)
            
filterfilepaths = glob.glob('/home/nfranz/.local/lib/anaconda3/lib/python3.11/site-packages/sedpy/data/filters/*.par')
filterfiles = {os.path.basename(path).split('.')[0] for path in filterfilepaths}

# use set difference to find missing files
set(filters) - filterfiles 

{'ps_g',
 'ps_i',
 'ps_r',
 'ps_y',
 'ps_z',
 'skymapper_g',
 'skymapper_i',
 'skymapper_z'}